In [11]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
# //import seaborn as sns
# sns.set_style("darkgrid")
data = pd.read_csv(r"D:\pdf's\6th sem\ML lab\ML-Lab-main\titanic.csv")
data['Embarked']=data['Embarked'].factorize()[0]

data = data[["Survived", "Pclass", "Age", "SibSp", "Parch", "Fare", 
"Embarked"]]
data.head(10)

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked
0,0,3,22.0,1,0,7.2500,0
1,1,1,38.0,1,0,71.2833,1
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,0
5,0,3,NaN,0,0,8.4583,2
6,0,1,54.0,0,0,51.8625,0
7,0,3,2.0,3,1,21.0750,0
8,1,3,27.0,0,2,11.1333,0
9,1,2,14.0,1,0,30.0708,1


In [15]:
def calculate_prior(df, Y):
 # classes = sorted(list(map(str, df[Y].unique())))
 classes = sorted(list(df[Y].unique()))
 prior = []
 for i in classes:
    prior.append(len(df[df[Y]==i])/len(df))
 return prior
 
def calculate_likelihood_gaussian(df, feat_name, feat_val, Y, label):
 # feat = list(df.columns)
 df = df[df[Y]==label]
 mean, std = df[feat_name].mean(), df[feat_name].std()
 p_x_given_y = (1 / (np.sqrt(2 * np.pi) * std)) * np.exp(-((feat_val-mean)**2 / (2 * std**2 )))

 return p_x_given_y



def naive_bayes_gaussian(df, X, Y):
 # get feature names
 features = list(df.columns)[:-1]
 # calculate prior
 prior = calculate_prior(df, Y)
 Y_pred = []
 # loop over every data sample
 for x in X:
 # calculate likelihood
    labels = sorted(list(df[Y].unique()))
    likelihood = [1]*len(labels)
    for j in range(len(labels)):
        for i in range(len(features)):
            likelihood[j] *= calculate_likelihood_gaussian(df,features[i], x[i], Y, labels[j]) 
                                                                                                                                                                                
 # calculate posterior probability (numerator only)
    post_prob = [1]*len(labels)
    for j in range(len(labels)):
        post_prob[j] = likelihood[j] * prior[j]
    Y_pred.append(np.argmax(post_prob))
 return np.array(Y_pred) 



train, test = train_test_split(data, test_size=.2, random_state=41)
X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values
Y_pred = naive_bayes_gaussian(train, X=X_test, Y="Embarked")

print(confusion_matrix(Y_test, Y_pred))
print(f1_score(Y_test, Y_pred,average="micro"))

[[  0   1   0   0]
 [  0 130   0   0]
 [  0  35   0   0]
 [  0  13   0   0]]
0.7262569832402235
